In [1]:
import csv
from datetime import datetime, timedelta
import random

def generate_weather_data(num_days=365):
    # Start date
    start_date = datetime(2023, 1, 1)
    
    # Generate data
    data = []
    for i in range(num_days):
        date = start_date + timedelta(days=i)
        # Generate temperature with seasonal variation
        base_temp = 20  # Base temperature
        seasonal_variation = 10 * math.sin(2 * math.pi * i / 365)  # Annual cycle
        random_variation = random.uniform(-3, 3)  # Daily random variation
        temperature = base_temp + seasonal_variation + random_variation
        
        data.append({
            'date': date.strftime('%Y-%m-%d'),
            'temperature': round(temperature, 2),
            'humidity': random.randint(40, 90)
        })
    
    # Write to CSV
    with open('weather_data.csv', 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['date', 'temperature', 'humidity'])
        writer.writeheader()
        writer.writerows(data)

if __name__ == '__main__':
    generate_weather_data()

NameError: name 'math' is not defined

In [ ]:
import csv

def read_csv_data(filename):
    temperatures = []
    humidity_values = []
    
    with open(filename) as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header row
        for row in reader:
            try:
                temp = float(row[1])
                humidity = float(row[2])
                temperatures.append(temp)
                humidity_values.append(humidity)
            except (ValueError, IndexError):
                print(f'Skipping invalid row: {row}')
                continue
    
    return temperatures, humidity_values


def calculate_mean(numbers):
    s = sum(numbers)
    N = len(numbers)
    return s/N

def calculate_median(numbers):
    N = len(numbers)
    numbers.sort()
    if N % 2 == 0:
        m1 = int(N/2) - 1
        m2 = int(N/2)
        median = (numbers[m1] + numbers[m2])/2
    else:
        m = int((N+1)/2) - 1
        median = numbers[m]
    return median